In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F

# Hyperparameters
batch_size = 128
learning_rate = 0.001
num_epochs = 10

# Data augmentation and preprocessing
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
def train(model, train_loader, criterion, optimizer, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        if i % 100 == 99:  # Print every 100 batches
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}, Accuracy: {100*correct/total:.2f}%')
            running_loss = 0.0

# Evaluating the model
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    print(f'Test Accuracy: {100*correct/total:.2f}%')
    return 100 * correct / total

# Training and evaluation loop
best_accuracy = 0
for epoch in range(num_epochs):
    train(model, train_loader, criterion, optimizer, epoch)
    accuracy = test(model, test_loader)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), 'best_model.pth')

print(f'Best Test Accuracy: {best_accuracy:.2f}%')

# Plot some test images with predictions
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Get some random test images
dataiter = iter(test_loader)
images, labels = dataiter.next()

# Print images
imshow(torchvision.utils.make_grid(images))
outputs = model(images.to(device))
_, predicted = outputs.max(1)

# Print predicted labels
print('Predicted: ', ' '.join(f'{train_dataset.classes[predicted[j]]}' for j in range(4)))

# Print actual labels
print('GroundTruth: ', ' '.join(f'{train_dataset.classes[labels[j]]}' for j in range(4)))


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/10], Step [100/391], Loss: 1.6993, Accuracy: 38.97%
Epoch [1/10], Step [200/391], Loss: 1.3814, Accuracy: 44.66%
Epoch [1/10], Step [300/391], Loss: 1.2454, Accuracy: 48.32%
Test Accuracy: 59.30%
Epoch [2/10], Step [100/391], Loss: 1.0703, Accuracy: 61.90%
Epoch [2/10], Step [200/391], Loss: 1.0380, Accuracy: 62.85%
Epoch [2/10], Step [300/391], Loss: 0.9977, Accuracy: 63.34%
Test Accuracy: 67.03%
Epoch [3/10], Step [100/391], Loss: 0.9009, Accuracy: 67.91%
Epoch [3/10], Step [200/391], Loss: 0.8898, Accuracy: 68.36%
Epoch [3/10], Step [300/391], Loss: 0.8618, Accuracy: 68.82%
Test Accuracy: 69.09%
Epoch [4/10], Step [100/391], Loss: 0.7920, Accuracy: 71.46%
Epoch [4/10], Step [200/391], Loss: 0.7621, Accuracy: 72.56%
Epoch [4/10], Step [300/391], Loss: 0.7769, Accuracy: 72.61%
Test Accuracy: 72.00%
Epoch [5/10], Step [100/391], Loss: 0.6969, Accuracy: 75.55%
Epoch [5/10], Step [200/391], Loss: 0.7100,

AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute 'next'